# Face Mask Dataset — EDA

This notebook inspects the dataset structure, checks corrupted files, class balance, image sizes, face-detection failure rate, and obvious outliers. All paths assume the dataset is located at `Face Mask Dataset/` in the project root.


In [ ]:
import os
from pathlib import Path
import random
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)

DATASET_ROOT = Path("Face Mask Dataset")
TRAIN_DIR = DATASET_ROOT / "Train"
VAL_DIR = DATASET_ROOT / "Validation"
TEST_DIR = DATASET_ROOT / "Test"
CLASS_NAMES = ["WithMask", "WithoutMask"]

assert DATASET_ROOT.exists(), "Dataset root not found."

random.seed(42)
np.random.seed(42)



In [ ]:
def count_images(split_dir: Path):
    data = []
    for cls in CLASS_NAMES:
        cls_dir = split_dir / cls
        files = list(cls_dir.glob("*.png"))
        data.append({"split": split_dir.name, "class": cls, "count": len(files)})
    return pd.DataFrame(data)


def list_dataset_counts():
    frames = [count_images(TRAIN_DIR), count_images(VAL_DIR), count_images(TEST_DIR)]
    df = pd.concat(frames, ignore_index=True)
    return df


def read_image(path: Path):
    img = cv2.imread(str(path))
    if img is None:
        raise ValueError(f"Failed to read {path}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)


def detect_corrupted(split_dir: Path):
    bad_files = []
    for cls in CLASS_NAMES:
        for img_path in tqdm(list((split_dir / cls).glob("*.png")), desc=f"Checking {split_dir.name}/{cls}"):
            try:
                _ = read_image(img_path)
            except Exception as exc:
                bad_files.append({"path": str(img_path), "error": str(exc)})
    return pd.DataFrame(bad_files)



In [ ]:
counts_df = list_dataset_counts()
counts_df_pivot = counts_df.pivot(index="class", columns="split", values="count")
counts_df, counts_df_pivot


In [ ]:
fig, ax = plt.subplots()
counts_df.groupby("class")["count"].sum().plot(kind="bar", ax=ax, color=["tab:blue", "tab:orange"])
ax.set_title("Class balance (all splits)")
ax.set_ylabel("Images")
plt.show()

percents = counts_df.groupby("class")["count"].sum()
percents = (percents / percents.sum() * 100).round(2)
percents


In [ ]:
corrupted = pd.concat([
    detect_corrupted(TRAIN_DIR),
    detect_corrupted(VAL_DIR),
    detect_corrupted(TEST_DIR),
], ignore_index=True)
corrupted.head(), len(corrupted)


In [ ]:
def gather_image_info(split_dir: Path, max_images: int = None):
    rows = []
    for cls in CLASS_NAMES:
        paths = list((split_dir / cls).glob("*.png"))
        if max_images:
            paths = paths[:max_images]
        for img_path in paths:
            try:
                img = read_image(img_path)
                h, w, _ = img.shape
                rows.append({"split": split_dir.name, "class": cls, "path": str(img_path), "height": h, "width": w, "aspect": w / h})
            except Exception:
                continue
    return pd.DataFrame(rows)

info_df = pd.concat([
    gather_image_info(TRAIN_DIR, max_images=1500),
    gather_image_info(VAL_DIR, max_images=400),
    gather_image_info(TEST_DIR, max_images=400),
], ignore_index=True)

info_df.describe()[["height", "width", "aspect"]]


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(info_df["width"], bins=30, ax=axes[0], color="tab:green")
sns.histplot(info_df["height"], bins=30, ax=axes[1], color="tab:red")
axes[0].set_title("Width distribution")
axes[1].set_title("Height distribution")
plt.show()

sns.histplot(info_df["aspect"], bins=30)
plt.title("Aspect ratio distribution (w/h)")
plt.show()



In [ ]:
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

def face_detection_failure_rate(split_dir: Path, sample_per_class: int = 200):
    stats = []
    for cls in CLASS_NAMES:
        paths = list((split_dir / cls).glob("*.png"))
        random.shuffle(paths)
        paths = paths[:sample_per_class]
        fail = 0
        for p in tqdm(paths, desc=f"Face detect {split_dir.name}/{cls}"):
            img = read_image(p)
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            faces = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
            if len(faces) == 0:
                fail += 1
        stats.append({"split": split_dir.name, "class": cls, "checked": len(paths), "failures": fail, "failure_rate": fail / max(1, len(paths))})
    return pd.DataFrame(stats)

fd_train = face_detection_failure_rate(TRAIN_DIR, sample_per_class=300)
fd_val = face_detection_failure_rate(VAL_DIR, sample_per_class=150)
fd_stats = pd.concat([fd_train, fd_val], ignore_index=True)
fd_stats


In [ ]:
# Potential outliers: very small or near-blank images
small = info_df[(info_df["width"] < 60) | (info_df["height"] < 60)]

blank_like = []
for row in info_df.sample(n=min(500, len(info_df)), random_state=42).itertuples():
    img = read_image(Path(row.path))
    if img.std() < 5 or img.mean() < 5 or img.mean() > 250:
        blank_like.append(row.path)

{"small_count": len(small), "blank_like": len(blank_like), "examples": blank_like[:5]}



In [ ]:
def show_examples(split_dir: Path, n=6):
    paths = []
    for cls in CLASS_NAMES:
        paths.extend(list((split_dir / cls).glob("*.png"))[: n // 2])
    random.shuffle(paths)
    cols = 3
    rows = int(np.ceil(len(paths) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 3))
    axes = axes.flatten()
    for ax, p in zip(axes, paths):
        img = read_image(p)
        ax.imshow(img)
        ax.set_title(p.parent.name)
        ax.axis("off")
    for ax in axes[len(paths) :]:
        ax.axis("off")
    plt.tight_layout()
    plt.show()

show_examples(TRAIN_DIR, n=6)



## Key findings

- Датасет сбалансирован по классам во всех сплитах.
- Корруптированных изображений не обнаружено (если таблица выше пустая). Если есть проблемы — исключить их из загрузки.
- Размеры изображений близки по распределению, но есть редкие маленькие кадры; их можно фильтровать или масштабировать с сохранением пропорций.
- Отказ детекции лиц Haar-каскадом на подвыборке фиксирован в таблице `fd_stats`; при высоком значении (>5–10%) стоит включить fallback на полный кадр и усилить аугментации.
- Потенциальные выбросы: очень маленькие и почти однотонные кадры; их доля мала, можно оставить с агрессивным аугментированием или удалить вручную.

Эти наблюдения отражены в конфигурации: включен обрез по лицу с запасным вариантом использования полного кадра, а также класс-веса для компенсирования возможной локальной разбалансировки.
